In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3 as sql

In [2]:
path = 'data/database.sqlite'
con = sql.connect(path)

In [3]:
# Let's read all available tables and explore what we got
country = pd.read_sql('select * from Country;', con)
league = pd.read_sql('select * from League;', con)
player = pd.read_sql('select * from Player;', con)
player_attributes = pd.read_sql('select * from Player_Attributes;', con)

In [4]:
league_country = league.merge(country, on = 'id')
league_country = league_country.rename(columns = {'name_x' : 'league', 'name_y' : 'country'})
league_country = league_country.drop('id', axis = 1)

In [5]:
league_country

,country_id,league,country
0,1,Belgium Jupiler League,Belgium
1,1729,England Premier League,England
2,4769,France Ligue 1,France
3,7809,Germany 1. Bundesliga,Germany
4,10257,Italy Serie A,Italy
5,13274,Netherlands Eredivisie,Netherlands
6,15722,Poland Ekstraklasa,Poland
7,17642,Portugal Liga ZON Sagres,Portugal
8,19694,Scotland Premier League,Scotland
9,21518,Spain LIGA BBVA,Spain


In [6]:
best_players = pd.read_excel('data/best_players.xlsx')

In [7]:
player_best = player[player.player_name.isin(best_players.name.to_list())]

2 names more are present

Let's find and remove the extra players from the player_best table

In [8]:
duplicate_players = player_best.player_name[player_best.player_name.duplicated()].to_list()
best_players[best_players.name.isin(duplicate_players)]

,season,name,club,champion,league
17,2009–10,Lisandro Lopez,Lyon,0,France Ligue 1
48,2008–9,Bruno Alves,Porto,1,Portugal Liga ZON Sagres


In [9]:
player_best[player_best.player_name.isin(duplicate_players)]

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
1406,1409,25920,Bruno Alves,138110,1981-11-27 00:00:00,187.96,183
1407,1410,375782,Bruno Alves,210292,1990-06-09 00:00:00,177.80,165
6177,6184,182456,Lisandro Lopez,215051,1989-09-01 00:00:00,187.96,176
6178,6185,30536,Lisandro Lopez,142707,1983-03-02 00:00:00,175.26,163


Lisandro Lopez, best player for 2009-10 season in Ligue 1 is born in 1983, source: https://en.wikipedia.org/wiki/Lisandro_L%C3%B3pez_(footballer,_born_1983)

Bruno Alves, best player for the 2008-09 season in Portugal Liga ZON Sagres is born in 1981, source:https://en.wikipedia.org/wiki/Bruno_Alves

In [10]:
player_best = player_best.drop(index = [1407, 6177])

In [11]:
best_players = best_players.merge(player_best, left_on = 'name', right_on = 'player_name')

best_players.set_index('id')[['player_api_id', 
                              'player_fifa_api_id', 
                              'name', 'season', 
                              'birthday', 'height', 
                              'weight', 'league', 
                              'champion']]
best_players.birthday = pd.to_datetime(best_players.birthday)

In [12]:
best_players.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72 entries, 0 to 71
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   season              72 non-null     object        
 1   name                72 non-null     object        
 2   club                72 non-null     object        
 3   champion            72 non-null     int64         
 4   league              72 non-null     object        
 5   id                  72 non-null     int64         
 6   player_api_id       72 non-null     int64         
 7   player_name         72 non-null     object        
 8   player_fifa_api_id  72 non-null     int64         
 9   birthday            72 non-null     datetime64[ns]
 10  height              72 non-null     float64       
 11  weight              72 non-null     int64         
dtypes: datetime64[ns](1), float64(1), int64(5), object(5)
memory usage: 7.3+ KB


In [13]:
player_attributes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183978 entries, 0 to 183977
Data columns (total 42 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   183978 non-null  int64  
 1   player_fifa_api_id   183978 non-null  int64  
 2   player_api_id        183978 non-null  int64  
 3   date                 183978 non-null  object 
 4   overall_rating       183142 non-null  float64
 5   potential            183142 non-null  float64
 6   preferred_foot       183142 non-null  object 
 7   attacking_work_rate  180748 non-null  object 
 8   defensive_work_rate  183142 non-null  object 
 9   crossing             183142 non-null  float64
 10  finishing            183142 non-null  float64
 11  heading_accuracy     183142 non-null  float64
 12  short_passing        183142 non-null  float64
 13  volleys              181265 non-null  float64
 14  dribbling            183142 non-null  float64
 15  curve            

In [14]:
player_attributes.date = pd.to_datetime(player_attributes.date)

Now, let's make a 'season' column based on the date feature

New season usually starts ~July, so

In [15]:
#player_attributes[player_attributes.date.dt.month > 6].date.dt.year.to_list(),
type(player_attributes[player_attributes.date.dt.month < 7].date.dt.year - 1)

pandas.core.series.Series

In [16]:
player_attributes.date.dt.year.value_counts()

2013    38867
2014    32808
2015    31834
2007    16138
2016    14103
2012    12632
2011    11976
2010    10829
2009     8993
2008     5798
Name: date, dtype: int64

In [17]:
player_attributes['season'] = player_attributes.date

In [18]:
player_attributes.season

0        2016-02-18
1        2015-11-19
2        2015-09-21
3        2015-03-20
4        2007-02-22
            ...    
183973   2009-08-30
183974   2009-02-22
183975   2008-08-30
183976   2007-08-30
183977   2007-02-22
Name: season, Length: 183978, dtype: datetime64[ns]

In [19]:
np.where(player_attributes.season.dt.month > 6, player_attributes.season.dt.year, player_attributes.season.dt.year - 1)

array([2015, 2015, 2015, ..., 2008, 2007, 2006], dtype=int64)

In [20]:
player_attributes.season.value_counts()

2007-02-22    11794
2013-09-20     6543
2011-08-30     6525
2015-09-21     6522
2012-08-31     6495
              ...  
2014-11-26        6
2015-09-10        5
2015-09-01        5
2016-02-13        1
2014-07-20        1
Name: season, Length: 197, dtype: int64

In [21]:
player_attributes.season.dt.year

0         2016
1         2015
2         2015
3         2015
4         2007
          ... 
183973    2009
183974    2009
183975    2008
183976    2007
183977    2007
Name: season, Length: 183978, dtype: int64

In [22]:
player_attributes.season = player_attributes.season.dt.year.astype('str').str.cat((player_attributes.season.dt.year - 1999).astype('str'), sep = '–')

In [23]:
player_attributes.season

0         2016–17
1         2015–16
2         2015–16
3         2015–16
4          2007–8
           ...   
183973    2009–10
183974    2009–10
183975     2008–9
183976     2007–8
183977     2007–8
Name: season, Length: 183978, dtype: object

In [ ]:
def seasonize_dates(df):
    
    #Create the season column
    df['season'] = player_attributes.date
    
    #Equate to the current year where date is after June (e.g. 2015-9-12 is for season 2015-16)
    #Equate to the previous year where date is before June (e.g. 2011-2-18 is for season 2010-11)
    #In this way we only need to add the year after to all records to finish the seasonizing of the dates
    df.season = np.where(df.season.dt.month > 6, df.season.dt.year, df.season.dt.year - 1)
    
    #Add the next year as string to the column (e.g. if 2015 = "2015-(2015 - 1999)" =  "2015-16")
    df.season = df.season.astype('str').str.cat((df.season - 1999).astype('str'), sep = '–')
    
    return df

In [ ]:
player_attributes = seasonize_dates(player_attributes)

Now we can filter by both `player_id` and `season`

In [ ]:
player_attributes

In [ ]:
best_players

In [ ]:
best_players.league.unique()

In [ ]:
best_players.groupby(best_players.season).first()

In [ ]:
pl_best_players = best_players[best_players.league == 'England Premier League']

In [ ]:
pl_best_players

In [ ]:
player_attributes = player_attributes.drop_duplicates(subset = ['season', 'player_api_id'])

In [ ]:
player_attributes

In [ ]:
pl_best_players.merge(player_attributes, how = 'left', on = ['season', 'player_api_id'])

In [ ]:
player_attributes[player_attributes.season == player_attributes.season.sort_values().unique()[2]]

In [ ]:
player_attributes.loc[(player_attributes.season == '2008-9') & (player_attributes.id == 7846)]

In [ ]:
player_attributes.loc[np.where((player_attributes.season == '2008-9') & (player_attributes.player_api_id == 30865))]